<a href="https://colab.research.google.com/github/Lucy-Moctezuma/ML-Tutorial-for-Antibiotic-Resistance-Predictions-for-E.-Coli/blob/main/6)_Making_Final_Metrics_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(**Note:**
Click on the button that reads *“Open in Colab”* to open this code in Google Colab. Once open in Google Colab, you can make a copy of the notebook in your personal drive and run the code by clicking a little triangle/arrow to the left of each code block.)

# **Creating Graphs for Accuracies, Recall and Precision Scores**

In this Notebook we will create a graph that will let us show the Accuracy,  Recall and Precision Scores for both Resistant (R) and Susceptibility (S). The figures we will create are inspired by the paper where our dataset comes from:

**Moradigaravand D, Palm M, Farewell A, Mustonen V, Warringer J, Parts L (2018) Prediction of antibiotic resistance in Escherichia coli from large-scale pan-genome data. PLoS Comput Biol 14(12): e1006258. https://doi.org/10.1371/journal.pcbi.1006258**

Below is the figure the author of the paper created:

![Moradigaravand graph](https://drive.google.com/uc?export=view&id=11aByLgM7pVPfBfhVU2LNBW2ZbZGbSgJH)

We will be creating a similar ones except instead of F1-scores we will focus on the **Recall** and **Precision** scores for each (Resistance and Susceptibility). In addition, our dataset includes an antibiotic drug that was not included in their original publication ("TZP") and we will not be using population structure as a train feature.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **1) Importing packages needed for visualization**

Below we will only be importing libraries needed for visualization of the results from all our models.

In [ ]:
# Data Wrangling Imports
import pandas as pd
import numpy as np

# Data visualization Imports
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch

# File Manipulation Imports
import os
from google.colab import drive
drive.mount('/content/drive/')

### **2) Loading each models scores and join them into a single dictionary**

Below we will import the 4 csv files created from Notebooks 2 to 5, where the metrics (Accuracy, Resistant Recall, Susceptibility Recall, Resistant Precision and Susceptibility Precision) from every combination of features (Y, G and GY) were stored. Then we will store it in a dictionary to access later on.

In [ ]:
# Loading all models metrics and joint them in a dictionary:
filepath = "/content/drive/MyDrive/EColi_ML_CSV_files/"
Model_Scores = {}

# Loading all data frames
LG_metrics = pd.read_csv(filepath+"LG_metrics_df.csv")
RF_metrics = pd.read_csv(filepath+"RF_metrics_df.csv")
GB_metrics = pd.read_csv(filepath+"GB_metrics_df.csv")
DL_metrics = pd.read_csv(filepath+"NN_metrics_df.csv")

# Adding all dataframes to dictionary
Model_Scores["Logistic_Regression"] = LG_metrics
Model_Scores["Random_Forest"] = RF_metrics
Model_Scores["Gradient_Boosted_Trees"] = GB_metrics
Model_Scores["Neural_Network"] = DL_metrics

Below is an example of how we can access a particular dataframe within our Python dictionary:

In [ ]:
# showing example of how a single dataframe can be accessed
Model_Scores['Gradient_Boosted_Trees'].head()

### **3) Select only the best scores for each drug from each model**

Below we will be creating a function that helps us extract the best metrics, we can choose to extract either Recall or Precision. By default we will work with Recall first as part of an example.

In [ ]:
# Extracting list of combos from dictionary created
combo_list = list(Model_Scores['Gradient_Boosted_Trees']["Drug_combo"].str.split("_", expand= True)[1].unique())
combo_list

In [ ]:
# Extracting list of drugs from dictionary created
drug_list = list(Model_Scores['Gradient_Boosted_Trees']["Drug_combo"].str[:3].unique())
drug_list

In [ ]:
# creating function that will take the best scores from each model
def Best_metrics(model,df):
  print("Selecting Best Scores for model: ",model)
  bestcores_dic = {}
  for drug in drug_list:
    data = df.loc[df["Drug_combo"].str.startswith(drug)]
    max_acc = max(data["Accuracy"])
    drug_combo = data["Drug_combo"][data["Accuracy"] == max_acc].unique()[0]
    R_rec = float(data[data["Drug_combo"] == drug_combo].iloc[:,2].iloc[0])
    S_rec = float(data[data["Drug_combo"] == drug_combo].iloc[:,3].iloc[0])
    R_prec = float(data[data["Drug_combo"] == drug_combo].iloc[:,4].iloc[0])
    S_prec = float(data[data["Drug_combo"] == drug_combo].iloc[:,5].iloc[0])
    bestcores_dic[drug_combo] = [max_acc, R_rec, S_rec, R_prec, S_prec]
    bestscores_df = pd.DataFrame.from_dict(bestcores_dic, orient ='index',columns=["Accuracy", "R_recall", "S_recall", "R_precision", "S_precision"]).reset_index()
    bestscores_df = bestscores_df.rename(columns = {'index':'Drug_combo'})
  return bestscores_df

Below we will implement this function to extract the best Accuracy and other metrics from our Logistic Regression Results.

In [ ]:
# Implementation of function Best_metrics() example with Logistic Regression scores
LG_best_metrics = Best_metrics("Logistic_Regression", Model_Scores["Logistic_Regression"])
LG_best_metrics

### **4) Converting combo strings into separate columns of G, Y and S**

The function below will help us code what feature datasets were used. This information will now appear as 2 columns, one per dataset feature (G and Y). These columns will help us create the black and white checkered grids we see at the bottom of the example figure from Moradigaravand's paper.


In [ ]:
# Creating a function that takes the combo strings from Drug_combo column and turns them into separate columns
def make_GY(bestscores_df):
  # Create 2 new columns one for each type of features (Gene presence, Year and Population structure)
  bestscores_df["G"] = " "
  bestscores_df["Y"] = " "

  # Read the combo part of Drug_combo
  split_c = bestscores_df["Drug_combo"].str.split("_", expand=True)
  i=0
  while i < len(split_c[1]):
    split_each_c = [x for x in split_c[1][i]]
    for g in split_each_c:
      if "G" in split_each_c:
        bestscores_df.at[i,"G"] = 1
      else:
        bestscores_df.at[i,"G"] = 0
    for y in split_each_c:
      if "Y" in split_each_c:
        bestscores_df.at[i,"Y"] = 1
      else:
        bestscores_df.at[i,"Y"] = 0
    i += 1
  bestscores_df["Drug_combo"] = bestscores_df["Drug_combo"].map(lambda x: x.rstrip('_GY'))
  bestscores_df.rename(columns={"Drug_combo": "Drug"}, inplace = True)

  return bestscores_df

Below in the implementation we can see our three columns appear, one for each type of feature. 1 indicates that a feature was used and 0 that it wasn't used. Below we see an example of these columns for our Logistic Regression model results.

In [ ]:
# Implementing function make_GY()
GY_LG_best_metrics = make_GY(LG_best_metrics)
GY_LG_best_metrics

### **5) Extracting best metrics and coding GYS into columns for all models**

Below we will be implementing the two functions we have created before: **Best_metrics()** and **make_GYS()** for all the results from each of the models.

In [ ]:
# Getting the best metrics for all the models
Best_metrics_models = {}
for model, df in Model_Scores.items():
  # select the best scores obtained from each model
  Model_best_metrics = Best_metrics(model, df)

  # Code GYS data in 0 "for absence" and 1 "for presence" into 3 columns
  GY_coded_best = make_GY(Model_best_metrics)
  print(GY_coded_best)

  # Save new dataframe in a dictionary with best metrics selected
  Best_metrics_models[model] = GY_coded_best

### **6) Creating a function to plot bar charts for each metric**

The function below will help us plot barplots comparing the metrics we choose for all the models created.

In [ ]:
def barplot(metric_col, subplot_axis, label_show = True):
  for model, df in Best_metrics_models.items():
    X_axis = np.arange(len(drug_list))
    X_labels = drug_list

    Y_axis = np.arange(0,1.2,0.2)

    subplot_axis.set_ylabel(metric_col, fontsize = 14)
    subplot_axis.set_ylim(bottom=0, top=1)

    subplot_axis.xaxis.set_ticklabels(X_labels, fontsize = 15)
    subplot_axis.xaxis.set_ticks(X_axis)
    subplot_axis.margins(x=0)

    if label_show == False:
      subplot_axis.tick_params(left = True, right = False , labelleft = True , labelbottom = False, bottom = True)

    if model == "Logistic_Regression":
      X_axis = X_axis - 0.3
      color = "plum"
      label = "LG"
      subplot_axis.bar(X_axis, list(df[metric_col]), width =.2, align = 'center', color = color, label = label, edgecolor="gray")
    elif model == "Random_Forest":
      X_axis = X_axis - 0.1
      color = "cadetblue"
      label = "RF"
      subplot_axis.bar(X_axis, list(df[metric_col]), width =.2, align = 'center', color = color, label = label, edgecolor="gray")
    elif model == "Gradient_Boosted_Trees":
      X_axis = X_axis + 0.1
      color = "goldenrod"
      label = "GB"
      subplot_axis.bar(X_axis, list(df[metric_col]), width =.2, align = 'center', color = color, label = label, edgecolor="gray")
    elif model == "Neural_Network":
      X_axis = X_axis + 0.3
      color = "steelblue"
      label = "GB"
      subplot_axis.bar(X_axis, list(df[metric_col]), width =.2, align = 'center', color = color, label = label, edgecolor="gray")
  return

Below we have the implementation of this function on the **Accuracy** metric. However we can choose other ones available as well. In this case we could choose **R_recall**, **S_precision**, etc.

In [ ]:
# implementing function barplot() for one metric "Accuracy"
figure, subs = plt.subplots(1, 1,figsize = (18,8))
acc_plot = barplot("Accuracy",subs,label_show=True)

### **7) Creating a function to plot GYS grids for each drug**

The function below will create a small graphic that will indicate what dataset was used (G, Y, and/or S) when training the model. This is made using the columns created before when we implemented the **make_GYS()** function.

In [ ]:
# function that creates a graph for each of the drugs
def GY_gridplot(drug,subplot_axis, label_show = True, title_pos = -0.2):
  # Create 2 new lists one for each type of features (Year and Gene presence)
  Y_list = [] # storing whether it used or not year data
  G_list = [] # storing whether it used or not accessory gene data

  # Fill up corresponding lists from drug results from each model
  for model, df in Best_metrics_models.items():
    for drug_name in df["Drug"]:
      if drug_name == drug:
        Y_list.append(int(df["Y"][df["Drug"]==drug].iloc[0]))
        G_list.append(int(df["G"][df["Drug"]==drug].iloc[0]))

  Drug_GY_list = [Y_list, G_list]

  plt.yticks(np.arange(2), ["Year", "Accessory Genes"])
  if label_show == False:
      subplot_axis.tick_params(left = False, labelleft = False)

  orig_map = plt.colormaps['gray']
  reversed_map = orig_map.reversed()
  subplot_axis.imshow(Drug_GY_list, cmap = reversed_map)

  subplot_axis.axvline(x=0.5)
  subplot_axis.axvline(x=1.5)
  subplot_axis.axvline(x=2.5)
  subplot_axis.axhline(y=0.5)
  subplot_axis.axhline(y=1.5)
  subplot_axis.set_title(drug, fontsize= 15, y=title_pos)
  subplot_axis.tick_params(
    axis = 'x',
    which = 'both',
    bottom = False,
    top = False,
    labelbottom = False)
  return [Y_list, G_list]

The way we read this gridplot is that if the square is black, then it means that the dataset feature was used. If it is white, then it was not used. In the example implementation below, we are choosing the antibiotic CTZ and have extracted their best metrics from each of the models.

In the first column of the graph we see year and Genes (Gene Presence and Absence data features) in black, meaning this was the combination that yielded the best results for one particular model. Each column represents one model.

In [ ]:
# implementing function GY_gridplot() for one drug "CTZ"
figure, subs = plt.subplots()
CTZ_gridplot = GY_gridplot("CTZ",subs, label_show=True)

### **8) Create the final composite graph (Bargraphs + GY gridplots)**

Below we finally create our Visualization graphs for the best metrics from all the models.

In [ ]:
# Code to create bargraphs
fig = plt.figure(figsize = (24,15), constrained_layout=False)

gs1 = fig.add_gridspec(nrows=6, ncols=12, left=0.05, right=0.6, wspace=0.07)
# Accuracy barcharts for all models
acc_axis = fig.add_subplot(gs1[0, :])
acc_axis.set_title('Prediction of Antibiotic Resistance From Pan-Genome Data', fontsize = 20)
acc_plot = barplot("Accuracy",acc_axis, label_show = False)

# R_recall_score barcharts for all models
R_metric_axis = fig.add_subplot(gs1[1, :])
R_metric_plot = barplot('R_recall',R_metric_axis, label_show = False)

# S_recall barcharts for all models
S_metric_axis = fig.add_subplot(gs1[2, :])
S_metric_plot = barplot('S_recall',S_metric_axis, label_show = False)

# R_precision barcharts for all models
R_metric_axis = fig.add_subplot(gs1[3, :])
R_metric_plot = barplot('R_precision',R_metric_axis, label_show = False)

# S_precision barcharts for all models
S_metric_axis = fig.add_subplot(gs1[4, :])
S_metric_plot = barplot('S_precision',S_metric_axis, label_show = False)

# GY gridplot charts for each drug
gs2 = fig.add_gridspec(nrows=1, ncols=12, top=0.44,bottom=0,left=0.05, right=0.6, wspace=0.2)
i=0
while (i< len(drug_list)):
  for drug in drug_list:
    if i == 0:
      Drug_grid = fig.add_subplot(gs2[-1, i])
      drug_GY = GY_gridplot(drug, Drug_grid, label_show=True, title_pos=-0.6)
      i+=1
    else:
      Drug_grid = fig.add_subplot(gs2[-1, i])
      drug_GY = GY_gridplot(drug, Drug_grid, label_show=False, title_pos=-0.6)
      i+=1

legend_elements = [Patch(facecolor='plum', edgecolor='plum', label='Logistic Regression'),
                   Patch(facecolor='cadetblue', edgecolor='cadetblue', label='Random Forest'),
                   Patch(facecolor='goldenrod', edgecolor='goldenrod', label='Extreme Gradient Boosted Tree'),
                   Patch(facecolor='steelblue', edgecolor='steelblue', label='Neural Network'),
                   Patch(facecolor='black', edgecolor="black", label='Used'),
                   Patch(facecolor='white', edgecolor="black",label='Not Used')]

plt.legend(handles=legend_elements,loc = 'lower center', bbox_to_anchor=(-6.1, -1), prop ={'size': 15}, borderaxespad=-3, ncol = 3)

# saving plot as a jpg image (Make sure to change names if you plan to create more than one!)
plt.savefig('/content/drive/MyDrive/EColi_ML_Plots/Final_Metrics_Plot.jpg',dpi=400, bbox_inches="tight")


The results show that, over all, the extreme gradient-boosted tree model does very well, and has the highest accuracy and recall for many of the drugs. We also see that gene presence-absence data are most likely to be included in the best-scoring models, whereas year of isolation is least likely to be included. Finally, we notice big differences between drugs and between resistance and susceptibility. This may be explained by the fact that for some of the drugs, the number of resistant strains is low, which makes it hard for the models to “learn” to recognize them.

It is also important to acknowledge that the graph only shows the best results using only regular K-fold validation, therefore it is left as a future exercise to recreate this graph using instead Blocked crossvalidation or another method where population structure is taken into account.

***Open Question:***

What differences do you expect to see in the final graph between random k-fold crossvalidation and Stratified blocked crossvalidation?
